In [82]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [83]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [84]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 54)]              0         
                                                                 
 dense_60 (Dense)            (None, 256)               14080     
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_61 (Dense)            (None, 128)               32896     
                                                                 
 dense_62 (Dense)            (None, 128)               16512     
                                                                 
 dense_63 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainabl

In [85]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [86]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss')
callbacks = [
    reduce_lr,
    tf.keras.callbacks.EarlyStopping(
        # Quantity to be monitored.
        monitor='accuracy',  # 'loss'/'val_loss'或accuracy/val_accuracy(根据metrics)
        # Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.
        min_delta=1e-3,
        # Number of epochs with no improvement after which training will be stopped.
        patience=3,
        # verbosity mode.
        verbose=1,
        # One of {"auto", "min", "max"}. In min mode, training will stop when the quantity monitored has stopped decreasing; in "max" mode it will stop when the quantity monitored has stopped increasing; in "auto" mode, the direction is automatically inferred from the name of the monitored quantity.
        mode='max')  # 由monitory='accuracy',故设置mode='max'

]
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),
          # List of keras.callbacks.Callback instances.
          callbacks=callbacks)

Epoch 1/50
141/141 [==============================] - 1s 4ms/step - loss: 1.4015 - accuracy: 0.4482 - val_loss: 1.1313 - val_accuracy: 0.5700 - lr: 0.0010
Epoch 2/50
141/141 [==============================] - 0s 3ms/step - loss: 1.2466 - accuracy: 0.4942 - val_loss: 1.1174 - val_accuracy: 0.5580 - 